*This notebook can be found on* [github](https://github.com/qojulia/QuantumOptics.jl-examples/tree/master/notebooks/wavepacket2D.ipynb)

# Dynamics of a two-dimensional wavepacket

In [ ]:
using QuantumOptics, PyPlot

In this example we will show how one can evolve a wavepacket in 2 spatial dimensions. We will do that using [`tensor`](@ref) products between the two spaces. We start similarly to the 1D case, by defining a position basis and a momentum operator for each dimension.

In [ ]:
Npoints = 50
Npointsy = 50

xmin = -30
xmax = 30
b_position = PositionBasis(xmin, xmax, Npoints)
b_momentum = MomentumBasis(b_position)

ymin = -20
ymax = 20
b_positiony = PositionBasis(ymin, ymax, Npointsy)
b_momentumy = MomentumBasis(b_positiony);

The collective `FFTOperator` is defined analogously to the 1D case using the composite bases.

In [ ]:
b_comp_x = b_position ⊗ b_positiony
b_comp_p = b_momentum ⊗ b_momentumy

Txp = transform(b_comp_x, b_comp_p)
Tpx = transform(b_comp_p, b_comp_x);

Thanks to these operators, we can specify the momentum operators in the respective `MomentumBasis`, where they are diagonal. Applying a diagonal operator is of course much more efficient.

In [ ]:
px = momentum(b_momentum)
py = momentum(b_momentumy);

Now that we have a composite basis, we can write each kinetic energy term in this composite basis. In order to keep the `FFTOperator` approach efficient, we will do this using lazy operations.

In [ ]:
Hkinx = LazyTensor(b_comp_p, [1, 2], [px^2/2, one(b_momentumy)])
Hkiny = LazyTensor(b_comp_p, [1, 2], [one(b_momentum), py^2/2])

Hkinx_FFT = LazyProduct(Txp, Hkinx, Tpx)
Hkiny_FFT = LazyProduct(Txp, Hkiny, Tpx);

Now we will add a two-dimensional potential. If we wanted to add a one-dimensional potential $V(x)$ the process is identical to the 1D case

In [ ]:
V0 = 4. # Height of Barrier
d = 5 # Width of Barrier
function V_barrier(x)
    if x < -d/2 || x > d/2
        return 0.
    else
        return V0
    end
end
V = potentialoperator(b_position, V_barrier)
# Re-write in 2D form:
V_comp = LazyTensor(b_comp, [1, 2], [V, one(b_positiony)])

However, making a truly 2D potential is a tiny bit more involved. The first thing you need to understand is that all that `potentialoperator` does is create a `diagonaloperator` (since the potential function is always a diagonal operator). So, considering the fact that the operators x and y are diagonal in a position basis (and hence also their tensor product), applying a function to a tensor product x ⊗ y results in a diagonal operator where the function is applied to each element of a diagonal.

This is easily done in the 2D case like


In [ ]:
potential(x,y) = sin(x*y) + cos(y)
Vvec = [potential(x, y) for x in xsample for y in ysample];

# V_comp = diagonaloperator(b_comp, Vvec) # we will use the 1D for visualization

Then one creates the full Hamiltonian simply by combining the kinetic and potential terms

In [ ]:
H = LazySum(Hkinx_comp, Hkiny_comp, V_comp)

Now we can perform the time evolution. Unfortunately `timeevolution.schroedinger` currently has no implementation that works with a LazyTensor operator consisting of LazyProduct operators (The issue tracking this is here: https://github.com/qojulia/QuantumOptics.jl/issues/190).

This means that the Hamiltonian matrix has to be recasted from Lazy form to "normal" form

In [ ]:
Hfull = sparse(full(H));

Now we will create a wavepacket in 2D and evolve it:

In [ ]:
ψx = gaussianstate(b_position, -10.0, 1.5, 2)
ψy = gaussianstate(b_positiony, 0, 0.5, 2)
ψ = ψx ⊗ ψy # again tensor product

T = collect(0.0:0.1:15.0)
tout, C = timeevolution.schroedinger(T, ψ, Hfull);

In [ ]:
c = C[1]
data = reshape(abs2.(c.data), (Npoints, Npointsy))'
norm = maximum(data)
function plot_wp(i)
    data = reshape(abs2.(C[i].data), (Npoints, Npointsy))'
    figure()
    ima = imshow(data, origin = "lower", extent = [xmin, xmax, ymin, ymax],
    cmap = "inferno", vmax = norm)
    tight_layout()
end
plot_wp(1)

In [ ]:
plot_wp(50)


In [ ]:
plot_wp(100)